In [ ]:
import os
# Run always
city = 'Vasteras'

input_dsm = r'C:\Users\akm03\Alaa\NERGYHUB\data\lantmateriet\{}\DSM\Laserdata_NH_2019'.format(city)
output_dsm = r'C:\Users\akm03\Alaa\NERGYHUB\data\lantmateriet\{}\DSM'.format(city)
city_dsm = os.path.join(output_dsm,'{}_dsm.tif'.format(city))

input_dtm = r'C:\Users\akm03\Alaa\NERGYHUB\data\lantmateriet\{}\DTM'.format(city)
city_dtm = os.path.join(input_dtm,'{}_dtm.tif'.format(city))

processed_path = 'C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/{0}/processed_data'.format(city)
buildings = os.path.join(processed_path,'buildings_filtered.shp')

In [ ]:
%get input_dsm output_dsm input_dtm processed_path --from Python3

### This code should be run in Python script editor in QGIS to convert laser data to raster, merge resulted tifs. add DSM and DTM values

#### For any help on qgis python algorithms, processing.algorithmHelp('gdal:rastercalculator')

In [ ]:
import os, processing, tempfile
# Change the city name or add a loop
input_wildcards = '*.laz'
step = 0.5
attribute = 0  # elevation
product = 0 # actual values
method = 3 # 1 highest; 0 lowest; 3 average; 4 stdev
filters = 2 # keep_first

output_raster_format = 0 # tif
cores = 4

las_params = {'INPUT_DIRECTORY': input_dsm, 
          'STEP': step, 
          'INPUT_WILDCARDS': input_wildcards, 
          'ATTRIBUTE': attribute, 
          'FILTER_RETURN_CLASS_FLAGS1': filters,
          'PRODUCT': product,
          'OUTPUT_DIRECTORY': output_dsm,
          'OUTPUT_RASTER_FORMAT': output_raster_format,
          'CORES': cores}

#processing.run("LAStools:las2demPro", las_params)

las_params = {'INPUT_DIRECTORY': input_dsm, 
          'STEP': step, 
          'INPUT_WILDCARDS': input_wildcards, 
          'ATTRIBUTE': attribute, 
          'METHOD': method,
          'OUTPUT_DIRECTORY': output_dsm,
          'OUTPUT_RASTER_FORMAT': output_raster_format,
          'CORES': cores}

processing.run("LAStools:lasgridPro", las_params)


## merge resulted rasters (it takes time 1410.86 seconds) 
rasters = [os.path.join(output_dsm,raster) for raster in os.listdir(output_dsm) if '.tif' in raster]
data_type = 5 # float 32 default
merge_params = {'INPUT': rasters,
                'DATA_TYPE': data_type,
                'OUTPUT': city_dsm}
processing.run("gdal:merge", merge_params)

## merge DTM rasters
rasters = [os.path.join(input_dtm,raster) for raster in os.listdir(input_dtm) if raster.endswith('.tif')]
data_type = 5 # float 32 default
merge_params = {'INPUT': rasters,
                'DATA_TYPE': data_type,
                'OUTPUT': city_dtm}
processing.run("gdal:merge", merge_params)

# clean buildings shapefile by removing polygons falling outside of the extent of the dtm


# ## get raster extents as temporary shapefile
#tf = tempfile.TemporaryDirectory()
dtm_extent = os.path.join(processed_path, '{}_dtm_extent.shp'.format(city))
round_to = 0
extent_params = {'INPUT': city_dtm,
                'OUTPUT': dtm_extent,
                'ROUND_TO': round_to}
processing.run("native:polygonfromlayerextent", extent_params)


# ## select only polygons within raster extent
raw_buildings = 'C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/Vasteras/Fastighetskartan/by_19.shp'
predicate = 6 # are within
method = 0
select_params = {'INPUT': raw_buildings,                 
                 'PREDICATE': predicate,
                 'INTERSECT': dtm_extent,
                 'METHOD': method}

processing.run("qgis:selectbylocation", select_params)


## Save selected features
save_params = {'INPUT':raw_buildings,
              'OUTPUT':buildings}
processing.run('qgis:saveselectedfeatures', save_params)

              
### add raster values to polygons
prefix = 'DSM_'
stats = [2, 3, 6] # mean and max
dsm_params = {'INPUT_RASTER': city_dsm, 
          'RASTER_BAND': 1, 
          'INPUT_VECTOR': buildings, 
          'COLUMN_PREFIX': prefix, 
          'STATISTICS': stats,
          'OUTPUT': buildings}
processing.run("qgis:zonalstatistics", dsm_params)  

### add raster values to polygons
prefix = 'DTM_'
stats = [2, 3, 6] # mean and max
dtm_params = {'INPUT_RASTER': city_dtm, 
          'RASTER_BAND': 1, 
          'INPUT_VECTOR': buildings, 
          'COLUMN_PREFIX': prefix, 
          'STATISTICS': stats,
          'OUTPUT': buildings}
processing.run("qgis:zonalstatistics", dtm_params)  


### This script is to check for geometrical duplicates

In [ ]:
import mmqgis
from mmqgis import mmqgis_library

layer_file = os.path.join(processed_path,'buildings_filtered.shp')
input_layer = QgsVectorLayer(layer_file)

# non duplicated features will be saved while duplicated will be deleted. the file can be geojson
output_file_name = os.path.join(processed_path,"no-duplicates.shp") 
message = mmqgis_library.mmqgis_delete_duplicate_geometries(input_layer, output_file_name)
print("Delete duplicate geometries: " + str(message))

# if message != None:    

### The following scripts are to get the height at the buildings perimeters

In [ ]:
import os, processing, tempfile
### Run the following code in qgis console to get the buildings buffers at the perimeters
distance = -0.1
buffered = os.path.join(processed_path,'buffered.shp')
buffer_params = {'INPUT': buildings,
                'DISTANCE': distance,
                'OUTPUT': buffered}
buffered_layer = processing.run("native:buffer", buffer_params) 

distance = -0.5
tf = tempfile.TemporaryDirectory()
buffered2 = tf.name + "\\buffered2.shp"
buffer_params = {'INPUT': buffered,
                'DISTANCE': distance,
                'OUTPUT': buffered2}
buffered_layer = processing.run("native:buffer", buffer_params) 


# calculate the difference
buildings_perimeters = os.path.join(processed_path, 'buildings_perimeters.shp')
difference_params = {'INPUT': buffered,
                    'OVERLAY': buffered2,
                    'OUTPUT': buildings_perimeters}
processing.run("native:difference", difference_params) 


### The following code to run here 

In [ ]:
library(raster)
library(rgdal)
library(qdapTools)
library(dplyr)
library(tmaptools)
library(stringr)
library(data.table)
library(sqldf)
library(DBI)

city <- 'Vasteras'
output_dsm <- sprintf('C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/%s/DSM/las2dempro',city)
processed_path <- sprintf('C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/%s/processed_data',city)
city_dsm <- file.path(output_dsm,sprintf('%s_dsm.tif',city))
buildings_perimeters <- file.path(processed_path, 'buildings_perimeters.shp')
#buildings_perimeters <- file.path(processed_path, 'buildings.shp')

raster <- stack(city_dsm)
poly <- readOGR(buildings_perimeters)

### Raster_values csv will be huge. Add to database

In [ ]:
db <- 'rasterDB'  #provide the name of your db
host_db <- 'localhost' #i.e. # i.e. 'ec2-54-83-201-96.compute-1.amazonaws.com'  
db_port <- '5432'  # or any other port specified by the DBA
db_user <- 'postgres'  
db_password <- '3n3rgY_8ub'
con <- dbConnect(RPostgres::Postgres(), dbname = db, host=host_db, port=db_port, user=db_user, password=db_password)

In [ ]:
valList <- list()
height <- list()
j = 1

sub_length <- 500

for(i in 1:length(poly)){
    #print(i)
    flush.console()
    building <- poly[i,]
    values <- extract(raster,building)
    df_values <- data.frame(ID=building$OBJEKT_ID,Value = values[[1]][,1])
    #df_values <- data.frame(ID=building$BuildingID,Value = values[[1]][,1])
    
    valList[[j]] <- df_values
    j <- j + 1

    subframe <- df_values[df_values$Value >= 2.2,]
    #subframe <- subframe[order(df_values$Value),]
    #subframe$Number <- 1:nrow(subframe)
    #subframe$Percentage <- subframe$Number/max(subframe$Number, na.rm = TRUE)
    #subframe <- subframe[subframe$Percentage<0.25,]
    #subframe <- subframe[, !names(subframe)%in% c('Number')]
    #rownames(subframe) <- 1:nrow(subframe)
    height[[i]] <- data.frame(ID=building$OBJEKT_ID, Value = median(subframe$Value, na.rm = TRUE))
    
    if (length(valList) == sub_length) {
        dbWriteTable(con, sprintf('raster_values_at_deeper_perimeters_%s',i), do.call(rbind.data.frame,valList),overwrite = T)
        #dbWriteTable(con, sprintf('raster_values_%s',i), do.call(rbind.data.frame,valList),overwrite = T)
        #fwrite(do.call(rbind.data.frame,valList),file.path(output_dsm,'at_buildings', sprintf("dsm_values_%s.csv", i)))
        
        valList <- list()
        j <- 1
    }
}

if (length(poly) %% sub_length != 0) {
    dbWriteTable(con, sprintf('raster_values_at_deeper_perimeters_%s',i), do.call(rbind.data.frame,valList),overwrite = T)
    #dbWriteTable(con, sprintf('raster_values_at_perimeters_%s',i), do.call(rbind.data.frame,valList),overwrite = T)
    #fwrite(do.call(rbind.data.frame,valList),file.path(output_dsm,'at_buildings', sprintf("dsm_values_%s.csv", i)))
}

fwrite(do.call(rbind.data.frame,height),file.path(output_dsm,'at_deeper_perimeters', "dsm_values_at_perimeters_3.csv"))